In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.signal import savgol_filter

# === 1. Load stage data ===
def load_stage_file(file_path):
    return pd.read_csv(file_path, comment="#")

gt_2023 = load_stage_file("/content/drive/MyDrive/CIROH_UWRL/BlacksmithFork/data_analysis/BSF_CONF_BA_SourceID_1_QC_0_Year_2023.csv")
gt_2024 = load_stage_file("/content/drive/MyDrive/CIROH_UWRL/BlacksmithFork/data_analysis/BSF_CONF_BA_SourceID_1_QC_0_Year_2024.csv")
gt_2025 = load_stage_file("/content/drive/MyDrive/CIROH_UWRL/BlacksmithFork/data_analysis/BSF_CONF_BA_SourceID_1_QC_0_Year_2025.csv")

# Combine and filter
gt_all = pd.concat([gt_2023, gt_2024, gt_2025], ignore_index=True)
gt_all["timestamp"] = pd.to_datetime(gt_all["DateTimeUTC"])
gt_all = gt_all[["timestamp", "Stage", "Discharge_cms"]]
gt_all = gt_all[gt_all["Stage"] != -9999.0]
gt_all = gt_all.rename(columns={"Discharge_cms": "Discharge"})

# === 2. Load ROI data ===
roi_df = pd.read_csv("/content/drive/MyDrive/CIROH_UWRL/BlacksmithFork/data_analysis/blacksmithcsv.csv")
roi_df["timestamp"] = pd.to_datetime(roi_df["image_timestamp"]).dt.round("15min")

# Extract ROI1 and ROI2
roi_vals = roi_df["roi"].str.extract(r"\{(\d+),(\d+)\}")
roi_df["roi1_pixels"] = pd.to_numeric(roi_vals[0], errors="coerce")
roi_df["roi2_pixels"] = pd.to_numeric(roi_vals[1], errors="coerce")
roi_df_clean = roi_df[["timestamp", "roi1_pixels", "roi2_pixels", "iou_score"]]

# === 3. Merge stage and ROI data ===
merged_df = pd.merge(gt_all, roi_df_clean, on="timestamp", how="left")

# === 4. Linear regression filling ===
train_df = merged_df.dropna(subset=["Stage", "roi1_pixels", "roi2_pixels"])
model_roi1 = LinearRegression().fit(train_df[["Stage"]], train_df["roi1_pixels"])
model_roi2 = LinearRegression().fit(train_df[["Stage"]], train_df["roi2_pixels"])

merged_df["roi1_filled"] = merged_df["roi1_pixels"]
merged_df["roi2_filled"] = merged_df["roi2_pixels"]

missing_roi1 = merged_df["roi1_pixels"].isna()
missing_roi2 = merged_df["roi2_pixels"].isna()

merged_df.loc[missing_roi1, "roi1_filled"] = model_roi1.predict(merged_df.loc[missing_roi1, ["Stage"]])
merged_df.loc[missing_roi2, "roi2_filled"] = model_roi2.predict(merged_df.loc[missing_roi2, ["Stage"]])

# === 5. Smooth filled ROI signals ===
def get_valid_window_length(series, desired=31):
    n = len(series)
    return desired if (n >= desired and desired % 2 == 1) else (n if n % 2 == 1 else n - 1)

window_roi1 = get_valid_window_length(merged_df["roi1_filled"].dropna(), 31)
window_roi2 = get_valid_window_length(merged_df["roi2_filled"].dropna(), 31)

merged_df["roi1_smoothed"] = savgol_filter(
    merged_df["roi1_filled"].fillna(method='ffill'), window_length=window_roi1, polyorder=2)
merged_df["roi2_smoothed"] = savgol_filter(
    merged_df["roi2_filled"].fillna(method='ffill'), window_length=window_roi2, polyorder=2)

#merged_df["Stage_ft"] = merged_df["Stage"] / 30.48  # 1 ft = 30.48 cm

start_date = "2023-12-04"
end_date = "2025-04-14"

filtered_df = merged_df[(merged_df["timestamp"] >= start_date) & (merged_df["timestamp"] <= end_date)]

In [ ]:
filtered_df.head()

In [ ]:
# === 6. Plot final results ===
fig, ax1 = plt.subplots(figsize=(14, 6))
ax1.plot(filtered_df["timestamp"], filtered_df["roi1_smoothed"], label="ROI1 Pixels", color="blue", alpha=0.8)
ax1.plot(filtered_df["timestamp"], filtered_df["roi2_smoothed"], label="ROI2 Pixels", color="green", alpha=0.8)
ax1.set_ylabel("ROI Pixel Values", fontsize=14)
ax1.set_xlabel("Image Timestamp", fontsize=14)
ax1.tick_params(axis='y', labelcolor='black', labelsize=12)
ax1.tick_params(axis='x', labelsize=12)
ax1.legend(loc="upper left", fontsize=12)

ax2 = ax1.twinx()
ax2.plot(filtered_df["timestamp"], filtered_df["Stage"], label="LRO Stage (cm)", color="red", linestyle="--", alpha=0.7)
ax2.set_ylabel("Stage (cm)", fontsize=14)
ax2.tick_params(axis='y', labelcolor='tab:red', labelsize=12)
ax2.legend(loc="upper right", fontsize=12)

plt.title("Time Series After IQR-Based Outlier Removal (ROI1 and ROI2 vs Stage)")
plt.grid(True)
plt.tight_layout()
plt.savefig("timeseries_stage.png", dpi=300)
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set style for aesthetics
sns.set(style="whitegrid")

# Plot IoU distribution (excluding NaN)
plt.figure(figsize=(10, 6))
sns.histplot(
    filtered_df["iou_score"].dropna(),
    bins=50,
    kde=True,
    color="skyblue",
    edgecolor="black"
)

plt.title("Distribution of IoU Scores", fontsize=16)
plt.xlabel("IoU Score", fontsize=14)
plt.ylabel("Frequency", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Subset the relevant columns
corr_df = filtered_df[["roi1_smoothed", "roi2_smoothed", "Stage"]].dropna()
corr_df.columns = ["ROI1", "ROI2", "Stage"]

# Pearson correlation
pearson_corr = corr_df.corr(method="pearson")
print("📌 Pearson Correlation Matrix:")
print(pearson_corr)

# Spearman correlation
spearman_corr = corr_df.corr(method="spearman")
print("\n📌 Spearman Correlation Matrix:")
print(spearman_corr)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Pearson
sns.heatmap(
    pearson_corr,
    annot=True,
    cmap="coolwarm",
    vmin=-1, vmax=1,
    ax=axes[0],
    annot_kws={"size": 12},  # annotation font size
    cbar_kws={"shrink": 0.8}
)
axes[0].set_title("Pearson Correlation", fontsize=14)
axes[0].tick_params(axis='both', labelsize=12)

# Spearman
sns.heatmap(
    spearman_corr,
    annot=True,
    cmap="coolwarm",
    vmin=-1, vmax=1,
    ax=axes[1],
    annot_kws={"size": 12},
    cbar_kws={"shrink": 0.8}
)
axes[1].set_title("Spearman Correlation", fontsize=14)
axes[1].tick_params(axis='both', labelsize=12)

plt.tight_layout()

# Save the figure
plt.savefig("correlation_stage.png", dpi=300, bbox_inches='tight')  # save as PNG

plt.show()

In [ ]:
# 1. Prepare IQR-filtered data
iqr_filtered_df = filtered_df[["roi1_smoothed", "roi2_smoothed", "Stage"]].dropna()
iqr_filtered_df.columns = ["ROI1 Pixel Count", "ROI2 Pixel Count", "Stage (cm)"]

# 2. EDA Summary
eda_summary = {
    "Total Entries": len(iqr_filtered_df),
    "ROI1 Range": (iqr_filtered_df["ROI1"].min(), iqr_filtered_df["ROI1"].max()),
    "ROI2 Range": (iqr_filtered_df["ROI2"].min(), iqr_filtered_df["ROI2"].max()),
    "Stage Range": (iqr_filtered_df["Stage"].min(), iqr_filtered_df["Stage"].max()),
    "Mean ROI1": iqr_filtered_df["ROI1"].mean(),
    "Mean ROI2": iqr_filtered_df["ROI2"].mean(),
    "Mean Stage": iqr_filtered_df["Stage"].mean(),
    "STD ROI1": iqr_filtered_df["ROI1"].std(),
    "STD ROI2": iqr_filtered_df["ROI2"].std(),
    "STD Stage": iqr_filtered_df["Stage"].std(),
    "Skewness ROI1": iqr_filtered_df["ROI1"].skew(),
    "Skewness ROI2": iqr_filtered_df["ROI2"].skew(),
    "Skewness Stage": iqr_filtered_df["Stage"].skew(),
    "Kurtosis ROI1": iqr_filtered_df["ROI1"].kurtosis(),
    "Kurtosis ROI2": iqr_filtered_df["ROI2"].kurtosis(),
    "Kurtosis Stage": iqr_filtered_df["Stage"].kurtosis()
}

# Print summary if desired
for k, v in eda_summary.items():
    print(f"{k}: {v}")

# 3. Pairplot
sns.set(style="whitegrid", context="notebook", font_scale=1.2)

# Plot the pairwise relationships
pairplot = sns.pairplot(
    iqr_filtered_df,
    kind="scatter",
    corner=True,
    diag_kind="kde",  # Use smooth KDE instead of hist
    plot_kws={"alpha": 0.5, "s": 20, "edgecolor": "k"},
    height=2.8,
    aspect=1.1,
    palette="muted"
)

# Set the main title
pairplot.fig.suptitle("Pairwise Feature Distributions and Relationships", fontsize=16, y=1.02)

# Improve axis label fonts
for ax in pairplot.axes.flat:
    if ax is not None:
        ax.tick_params(axis='both', labelsize=11)
        ax.xaxis.label.set_size(12)
        ax.yaxis.label.set_size(12)

# Remove unnecessary top/right spines for a cleaner look
sns.despine()

# Save and show
pairplot.savefig("pairwise_distribution.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    return df[(df[column] >= Q1 - 1.5 * IQR) & (df[column] <= Q3 + 1.5 * IQR)]

In [ ]:
# Fix: explicitly remove outliers on the correct column
iqr_filtered_df = filtered_df.copy()
iqr_filtered_df = remove_outliers_iqr(iqr_filtered_df, "roi1_smoothed")
iqr_filtered_df = remove_outliers_iqr(iqr_filtered_df, "roi2_smoothed")
iqr_filtered_df = remove_outliers_iqr(iqr_filtered_df, "Stage")

# Sanity fix: explicitly convert Stage_ft into a clean Stage column
iqr_filtered_df["Stage"] = iqr_filtered_df["Stage"].values  # overwrite with clean flat series

# Then drop the Stage_ft column if you want
#iqr_filtered_df = iqr_filtered_df.drop(columns=["Stage_ft"])

# Rename for plotting
iqr_filtered_df = iqr_filtered_df.rename(columns={
    "roi1_smoothed": "ROI1",
    "roi2_smoothed": "ROI2",
    "Stage": "Stage"
})

iqr_filtered_df = iqr_filtered_df.drop(columns=["ROI1"])

In [ ]:
# 2. EDA Summary
eda_summary = {
    "Total Entries": len(iqr_filtered_df),
    "ROI2 Range": (iqr_filtered_df["ROI2"].min(), iqr_filtered_df["ROI2"].max()),
    "Stage Range": (iqr_filtered_df["Stage"].min(), iqr_filtered_df["Stage"].max()),
    "Mean ROI2": iqr_filtered_df["ROI2"].mean(),
    "Mean Stage": iqr_filtered_df["Stage"].mean(),
    "STD ROI2": iqr_filtered_df["ROI2"].std(),
    "STD Stage": iqr_filtered_df["Stage"].std(),
    "Skewness ROI2": iqr_filtered_df["ROI2"].skew(),
    "Skewness Stage": iqr_filtered_df["Stage"].skew(),
    "Kurtosis ROI2": iqr_filtered_df["ROI2"].kurtosis(),
    "Kurtosis Stage": iqr_filtered_df["Stage"].kurtosis()
}

# Print summary if desired
for k, v in eda_summary.items():
    print(f"{k}: {v}")

# 3. Pairplot (fix the fig assignment)
sns.set_style("white")  # removes grid lines
# Rename for clear axis labeling
iqr_filtered_df_renamed = iqr_filtered_df.rename(columns={
    "ROI2": "ROI2 Pixel Counts",
    "Stage": "Stage (cm)"
})

# Create pairplot
pairplot = sns.pairplot(
    iqr_filtered_df_renamed[["ROI2 Pixel Counts", "Stage (cm)"]],
    corner=True,
    plot_kws={"alpha": 0.4, "s": 25},
    height=2.5
)

# Set title with larger font
pairplot.fig.suptitle("Pairwise Feature Distributions and Relationships", fontsize=14, y=1.03)

# Update font size for all axes in the pairplot
for ax in pairplot.axes.flat:
    if ax is not None:
        ax.grid(False)  # turns off grid
        sns.despine(ax=ax)  # removes top/right spines
        ax.tick_params(axis='both', labelsize=12)
        ax.xaxis.label.set_size(13)
        ax.yaxis.label.set_size(13)

# Save figure
pairplot.savefig("pairwise_distribution.png", dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# === Step 1: Prepare features and target ===
X = iqr_filtered_df[["ROI2"]]
y = iqr_filtered_df["Stage"]  # already cleaned and in feet
timestamps = iqr_filtered_df["timestamp"]

# === Step 2: Train-Test Split (80/20) ===
X_train, X_test, y_train, y_test, ts_train, ts_test = train_test_split(
    X, y, timestamps, test_size=0.2, random_state=42
)

# === Step 3: Fit model on training data ===
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# === Step 4: Predict on train and test sets ===
y_train_pred = lin_reg.predict(X_train)
y_test_pred = lin_reg.predict(X_test)

# === Step 5: Compute train-test metrics ===
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

# === Step 6: 5-Fold Cross-Validation ===
kf = KFold(n_splits=5, shuffle=True, random_state=42)

r2_scores = cross_val_score(lin_reg, X, y, cv=kf, scoring="r2")
neg_mse_scores = cross_val_score(lin_reg, X, y, cv=kf, scoring="neg_mean_squared_error")
rmse_scores = np.sqrt(-neg_mse_scores)

# === Step 7: Summarize results ===
results = {
    "Train R²": train_r2,
    "Test R²": test_r2,
    "Train RMSE": train_rmse,
    "Test RMSE": test_rmse,
    "Train MAE": train_mae,
    "Test MAE": test_mae,
    "CV R² Mean": np.mean(r2_scores),
    "CV R² Std": np.std(r2_scores),
    "CV RMSE Mean": np.mean(rmse_scores),
    "CV RMSE Std": np.std(rmse_scores),
}

# === Step 8: Display
for k, v in results.items():
    print(f"{k}: {v:.4f}")


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

# === Step 1: Polynomial Feature Expansion (Degree 2, ROI2 only) ===
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train[["ROI2"]])
X_test_poly = poly.transform(X_test[["ROI2"]])
X_full_poly = poly.transform(X[["ROI2"]])  # for cross-validation

# Optional: see generated features
# print(poly.get_feature_names_out(["ROI2"]))

# === Step 2: Train Model ===
model = LinearRegression()
model.fit(X_train_poly, y_train)

# === Step 3: Predictions ===
y_train_pred = model.predict(X_train_poly)
y_test_pred = model.predict(X_test_poly)

# === Step 4: Train-Test Metrics ===
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

# === Step 5: Cross-Validation ===
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_r2 = cross_val_score(model, X_full_poly, y, cv=kf, scoring="r2")
cv_neg_mse = cross_val_score(model, X_full_poly, y, cv=kf, scoring="neg_mean_squared_error")
cv_rmse = np.sqrt(-cv_neg_mse)

# === Step 6: Combine Metrics ===
metrics_poly = {
    "Train R²": train_r2,
    "Test R²": test_r2,
    "Train RMSE": train_rmse,
    "Test RMSE": test_rmse,
    "Train MAE": train_mae,
    "Test MAE": test_mae,
    "CV R² Mean": np.mean(cv_r2),
    "CV R² Std": np.std(cv_r2),
    "CV RMSE Mean": np.mean(cv_rmse),
    "CV RMSE Std": np.std(cv_rmse),
}

# === Step 7: Print Results Nicely ===
print("\n🔢 Polynomial Regression Results (Degree = 2, ROI2 only):")
for k, v in metrics_poly.items():
    print(f"{k}: {v:.4f}")


In [ ]:
print(y_train.shape)  # should be (n,)
print(type(y_train))

In [ ]:

# Define global axis limits with margin
buffer = 0.05 * (y.max() - y.min())
min_val = min(y.min(), y_train_pred.min(), y_test_pred.min()) - buffer
max_val = max(y.max(), y_train_pred.max(), y_test_pred.max()) + buffer

# Fit regression lines manually
train_slope, train_intercept = np.polyfit(y_train, y_train_pred, 1)
test_slope, test_intercept = np.polyfit(y_test, y_test_pred, 1)
x_line = np.linspace(min_val, max_val, 100)

plt.figure(figsize=(14, 6))

# Train set
plt.subplot(1, 2, 1)
plt.scatter(y_train, y_train_pred, alpha=0.6, color='dodgerblue', edgecolor='k', s=50, label='Predictions')
plt.plot(x_line, x_line, 'r--', linewidth=1.5, label='1:1 Line')
plt.plot(x_line, train_slope * x_line + train_intercept, 'b-', linewidth=2, label='Regression Line')
plt.title("Train Set: Actual vs Predicted Stage", fontsize=14)
plt.xlabel("Actual Stage (ft)", fontsize=12)
plt.ylabel("Predicted Stage (ft)", fontsize=12)
plt.xlim(min_val, max_val)
plt.ylim(min_val, max_val)
plt.grid(True)
plt.legend(fontsize=11)
plt.tick_params(labelsize=11)

# Test set
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_test_pred, alpha=0.6, color='forestgreen', edgecolor='k', s=50, label='Predictions')
plt.plot(x_line, x_line, 'r--', linewidth=1.5, label='1:1 Line')
plt.plot(x_line, test_slope * x_line + test_intercept, 'g-', linewidth=2, label='Regression Line')
plt.title("Test Set: Actual vs Predicted Stage", fontsize=14)
plt.xlabel("Actual Stage (ft)", fontsize=12)
plt.ylabel("Predicted Stage (ft)", fontsize=12)
plt.xlim(min_val, max_val)
plt.ylim(min_val, max_val)
plt.grid(True)
plt.legend(fontsize=11)
plt.tick_params(labelsize=11)

plt.tight_layout()

# Save the figure
plt.savefig("scatter_stage.png", dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# === Step 3: Tuned Random Forest Regressor ===
rf_tuned = RandomForestRegressor(
    n_estimators=100,
    max_depth=6,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42
)
rf_tuned.fit(X_train, y_train)

# === Step 4: Predictions ===
y_train_pred = rf_tuned.predict(X_train)
y_test_pred = rf_tuned.predict(X_test)

# === Step 5: Evaluation ===
metrics_rf = {
    "Train R²": r2_score(y_train, y_train_pred),
    "Test R²": r2_score(y_test, y_test_pred),
    "Train MAE": mean_absolute_error(y_train, y_train_pred),
    "Test MAE": mean_absolute_error(y_test, y_test_pred),
    "Train RMSE": np.sqrt(mean_squared_error(y_train, y_train_pred)),
    "Test RMSE": np.sqrt(mean_squared_error(y_test, y_test_pred)),
    "Train MSE": mean_squared_error(y_train, y_train_pred),
    "Test MSE": mean_squared_error(y_test, y_test_pred),
}

# === Step 6: Optional Cross-Validation (on full set) ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_r2 = cross_val_score(rf_tuned, X, y, cv=cv, scoring="r2")
cv_rmse = np.sqrt(-cross_val_score(rf_tuned, X, y, cv=cv, scoring="neg_mean_squared_error"))

metrics_rf.update({
    "CV R² Mean": np.mean(cv_r2),
    "CV R² Std": np.std(cv_r2),
    "CV RMSE Mean": np.mean(cv_rmse),
    "CV RMSE Std": np.std(cv_rmse),
})

# === Step 7: Display nicely ===
for k, v in metrics_rf.items():
    print(f"{k}: {v:.4f}")

In [ ]:
import xgboost as xgb

# === Step 1: Define XGBoost Regressor ===
xgb_model = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    subsample=0.9,
    colsample_bytree=0.9,
    objective='reg:squarederror',  # prevents warning for regression
    random_state=42
)

# === Step 2: Train on Train Set ===
xgb_model.fit(X_train, y_train)

# === Step 3: Predict ===
y_train_pred_xgb = xgb_model.predict(X_train)
y_test_pred_xgb = xgb_model.predict(X_test)

# === Step 4: Train-Test Evaluation ===
xgb_metrics = {
    "Train R²": r2_score(y_train, y_train_pred_xgb),
    "Test R²": r2_score(y_test, y_test_pred_xgb),
    "Train MAE": mean_absolute_error(y_train, y_train_pred_xgb),
    "Test MAE": mean_absolute_error(y_test, y_test_pred_xgb),
    "Train RMSE": np.sqrt(mean_squared_error(y_train, y_train_pred_xgb)),
    "Test RMSE": np.sqrt(mean_squared_error(y_test, y_test_pred_xgb)),
    "Train MSE": mean_squared_error(y_train, y_train_pred_xgb),
    "Test MSE": mean_squared_error(y_test, y_test_pred_xgb)
}

# === Step 5: Cross-Validation ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_r2 = cross_val_score(xgb_model, X, y, cv=cv, scoring="r2")
cv_rmse = np.sqrt(-cross_val_score(xgb_model, X, y, cv=cv, scoring="neg_mean_squared_error"))

xgb_metrics.update({
    "CV R² Mean": np.mean(cv_r2),
    "CV R² Std": np.std(cv_r2),
    "CV RMSE Mean": np.mean(cv_rmse),
    "CV RMSE Std": np.std(cv_rmse)
})

# === Step 6: Print Clean Summary ===
for k, v in xgb_metrics.items():
    print(f"{k}: {v:.4f}")

In [ ]:
# Define global axis limits with margin
buffer = 0.05 * (y.max() - y.min())
min_val = min(y.min(), y_train_pred.min(), y_test_pred.min()) - buffer
max_val = max(y.max(), y_train_pred.max(), y_test_pred.max()) + buffer

# Fit regression lines manually
train_slope, train_intercept = np.polyfit(y_train, y_train_pred, 1)
test_slope, test_intercept = np.polyfit(y_test, y_test_pred, 1)
x_line = np.linspace(min_val, max_val, 100)

# Format equations
train_eqn = f"y = {train_slope:.2f}x + {train_intercept:.2f}"
test_eqn = f"y = {test_slope:.2f}x + {test_intercept:.2f}"

plt.figure(figsize=(14, 6))

# Train set
plt.subplot(1, 2, 1)
plt.scatter(y_train, y_train_pred, alpha=0.6, color='dodgerblue', edgecolor='k', s=50, label='Predictions')
plt.plot(x_line, x_line, 'r--', linewidth=1.5, label='1:1 Line')
plt.plot(x_line, train_slope * x_line + train_intercept, 'b-', linewidth=2, label=f"Regression Line\n({train_eqn})")
plt.title("Train Set: Actual vs Predicted Stage", fontsize=14)
plt.xlabel("Actual Stage (cm)", fontsize=12)
plt.ylabel("Predicted Stage (cm)", fontsize=12)
plt.xlim(min_val, max_val)
plt.ylim(min_val, max_val)
plt.grid(True)
plt.legend(fontsize=11)
plt.tick_params(labelsize=11)

# Test set
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_test_pred, alpha=0.6, color='forestgreen', edgecolor='k', s=50, label='Predictions')
plt.plot(x_line, x_line, 'r--', linewidth=1.5, label='1:1 Line')
plt.plot(x_line, test_slope * x_line + test_intercept, 'g-', linewidth=2, label=f"Regression Line\n({test_eqn})")
plt.title("Test Set: Actual vs Predicted Stage", fontsize=14)
plt.xlabel("Actual Stage (cm)", fontsize=12)
plt.ylabel("Predicted Stage (cm)", fontsize=12)
plt.xlim(min_val, max_val)
plt.ylim(min_val, max_val)
plt.grid(True)
plt.legend(fontsize=11)
plt.tick_params(labelsize=11)

plt.tight_layout()
plt.savefig("scatter_stage.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
print(train_df.columns)

In [ ]:
import lightgbm as lgb

# === Step 1: Define LightGBM Regressor ===
lgb_model = lgb.LGBMRegressor(
    n_estimators=100,
    max_depth=6,
    num_leaves=31,
    learning_rate=0.1,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42
)

# === Step 2: Train ===
lgb_model.fit(X_train, y_train)

# === Step 3: Predict ===
y_train_pred_lgb = lgb_model.predict(X_train)
y_test_pred_lgb = lgb_model.predict(X_test)

# === Step 4: Evaluate on Train and Test ===
lgb_metrics = {
    "Train R²": r2_score(y_train, y_train_pred_lgb),
    "Test R²": r2_score(y_test, y_test_pred_lgb),
    "Train MAE": mean_absolute_error(y_train, y_train_pred_lgb),
    "Test MAE": mean_absolute_error(y_test, y_test_pred_lgb),
    "Train RMSE": np.sqrt(mean_squared_error(y_train, y_train_pred_lgb)),
    "Test RMSE": np.sqrt(mean_squared_error(y_test, y_test_pred_lgb)),
    "Train MSE": mean_squared_error(y_train, y_train_pred_lgb),
    "Test MSE": mean_squared_error(y_test, y_test_pred_lgb)
}

# === Step 5: Cross-Validation ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_r2 = cross_val_score(lgb_model, X, y, cv=cv, scoring="r2")
cv_rmse = np.sqrt(-cross_val_score(lgb_model, X, y, cv=cv, scoring="neg_mean_squared_error"))

lgb_metrics.update({
    "CV R² Mean": np.mean(cv_r2),
    "CV R² Std": np.std(cv_r2),
    "CV RMSE Mean": np.mean(cv_rmse),
    "CV RMSE Std": np.std(cv_rmse)
})

# === Step 6: Print Results ===
for k, v in lgb_metrics.items():
    print(f"{k}: {v:.4f}")

In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [ ]:
# === Step 2: Define SVR Pipeline ===
svr_model = make_pipeline(
    StandardScaler(),
    SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.01)
)

# === Step 3: Train and Predict ===
svr_model.fit(X_train, y_train)
y_train_pred_svr = svr_model.predict(X_train)
y_test_pred_svr = svr_model.predict(X_test)

# === Step 4: Evaluation Metrics ===
svr_metrics = {
    "Train R²": r2_score(y_train, y_train_pred_svr),
    "Test R²": r2_score(y_test, y_test_pred_svr),
    "Train MAE": mean_absolute_error(y_train, y_train_pred_svr),
    "Test MAE": mean_absolute_error(y_test, y_test_pred_svr),
    "Train RMSE": np.sqrt(mean_squared_error(y_train, y_train_pred_svr)),
    "Test RMSE": np.sqrt(mean_squared_error(y_test, y_test_pred_svr)),
    "Train MSE": mean_squared_error(y_train, y_train_pred_svr),
    "Test MSE": mean_squared_error(y_test, y_test_pred_svr)
}

# === Step 5: Cross-Validation ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_r2 = cross_val_score(svr_model, X, y, cv=cv, scoring="r2")
cv_rmse = np.sqrt(-cross_val_score(svr_model, X, y, cv=cv, scoring="neg_mean_squared_error"))

svr_metrics.update({
    "CV R² Mean": np.mean(cv_r2),
    "CV R² Std": np.std(cv_r2),
    "CV RMSE Mean": np.mean(cv_rmse),
    "CV RMSE Std": np.std(cv_rmse)
})

# === Step 6: Print Results ===
for k, v in svr_metrics.items():
    print(f"{k}: {v:.4f}")

In [ ]:
# === Step 1: Combine all model metrics into a dictionary ===
model_results = {
    #"SVR": svr_metrics,
    "Random Forest": metrics_rf,
    "XGBoost": xgb_metrics,
    "LightGBM": lgb_metrics,
    "Polynomial Regression (d=2)": metrics_poly,
    "Linear Regression": results
}

# === Step 2: Create DataFrame ===
ranking_df = pd.DataFrame(model_results).T[[
    "Test R²", "Test MAE", "Test RMSE", "CV R² Mean", "CV RMSE Mean"
]]
ranking_df.columns = ["R²", "MAE", "RMSE", "CV_R²", "CV_RMSE"]

# === Step 3: Rank models (lower rank is better) ===
ranking_df["R²_rank"] = ranking_df["R²"].rank(ascending=False)
ranking_df["MAE_rank"] = ranking_df["MAE"].rank(ascending=True)
ranking_df["RMSE_rank"] = ranking_df["RMSE"].rank(ascending=True)
ranking_df["CV_R²_rank"] = ranking_df["CV_R²"].rank(ascending=False)
ranking_df["CV_RMSE_rank"] = ranking_df["CV_RMSE"].rank(ascending=True)

# === Step 4: Calculate average rank and sort ===
ranking_df["Avg_Rank"] = ranking_df[
    ["R²_rank", "MAE_rank", "RMSE_rank", "CV_R²_rank", "CV_RMSE_rank"]
].mean(axis=1)

ranking_df = ranking_df.sort_values("Avg_Rank")

# Display or save
print(ranking_df[["R²", "MAE", "RMSE", "CV_R²", "CV_RMSE", "Avg_Rank"]])
# ranking_df.to_csv("model_ranking.csv", index=True)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

# === STEP 1: Predictions from the top 3 models ===
# Ensure these variables are already computed
pred_xgb = y_test_pred_xgb      # XGBoost
pred_lgb = y_test_pred_lgb      # LightGBM
pred_rf  = y_test_pred       # Random Forest

# === STEP 2: Create stacked combinations ===
stack_xgb_rf   = np.vstack([pred_xgb, pred_rf]).T
stack_xgb_lgb  = np.vstack([pred_xgb, pred_lgb]).T
stack_rf_lgb   = np.vstack([pred_rf, pred_lgb]).T
stack_all_3    = np.vstack([pred_xgb, pred_lgb, pred_rf]).T

# === STEP 3: Meta-model (stacked ensemble) predictions ===
stacked_xgb_rf   = LinearRegression().fit(stack_xgb_rf, y_test).predict(stack_xgb_rf)
stacked_xgb_lgb  = LinearRegression().fit(stack_xgb_lgb, y_test).predict(stack_xgb_lgb)
stacked_rf_lgb   = LinearRegression().fit(stack_rf_lgb, y_test).predict(stack_rf_lgb)
stacked_all_3    = LinearRegression().fit(stack_all_3, y_test).predict(stack_all_3)

# === STEP 4: Simple averages ===
avg_xgb_rf   = np.mean(stack_xgb_rf, axis=1)
avg_xgb_lgb  = np.mean(stack_xgb_lgb, axis=1)
avg_rf_lgb   = np.mean(stack_rf_lgb, axis=1)
avg_all_3    = np.mean(stack_all_3, axis=1)

# === STEP 5: Weighted average (customizable weights) ===
# You can tune these weights
weighted_avg = 0.4 * pred_rf + 0.4 * pred_lgb + 0.2 * pred_xgb

# === STEP 6: Evaluation function ===
def evaluate(y_true, y_pred):
    return {
        "R²": r2_score(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_true, y_pred))
    }

# === STEP 7: Compute metrics for all ensemble strategies ===
ensemble_results = {
    "Simple Avg (XGB + RF)": evaluate(y_test, avg_xgb_rf),
    "Simple Avg (XGB + LGB)": evaluate(y_test, avg_xgb_lgb),
    "Simple Avg (RF + LGB)": evaluate(y_test, avg_rf_lgb),
    "Simple Avg (All 3)": evaluate(y_test, avg_all_3),
    "Weighted Avg (0.4 RF + 0.4 LGB + 0.2 XGB)": evaluate(y_test, weighted_avg),
    "Stacked (XGB + RF)": evaluate(y_test, stacked_xgb_rf),
    "Stacked (XGB + LGB)": evaluate(y_test, stacked_xgb_lgb),
    "Stacked (RF + LGB)": evaluate(y_test, stacked_rf_lgb),
    "Stacked (All 3)": evaluate(y_test, stacked_all_3)
}

# === STEP 8: Convert to DataFrame and sort ===
ensemble_df = pd.DataFrame(ensemble_results).T
ensemble_df = ensemble_df.sort_values("R²", ascending=False)

# Display the final ensemble comparison
print(ensemble_df)

# Optional: Save to CSV
# ensemble_df.to_csv("ensemble_comparison_blacksmith.csv", index=True)


In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

# === BASE MODELS ===
base_models = {
    "LGBM": lgb_model,
    "Random Forest": rf_tuned,
    "XGBoost": xgb_model
}

# === PREPARE OOF PREDICTIONS ===
kf = KFold(n_splits=5, shuffle=True, random_state=42)

oof_preds = {name: np.zeros_like(y_train) for name in base_models}
test_preds = {name: [] for name in base_models}

# === 1. Generate OOF for training and test predictions ===
for train_idx, val_idx in kf.split(X_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    for name, model in base_models.items():
        model.fit(X_tr, y_tr)
        oof_preds[name][val_idx] = model.predict(X_val)
        test_preds[name].append(model.predict(X_test))  # accumulate fold predictions

# === 2. Stack into new training and test matrices ===
X_meta_train = np.vstack([oof_preds[name] for name in base_models]).T
X_meta_test = np.mean([np.vstack(test_preds[name]) for name in base_models], axis=1).T

# === 3. Train meta-model on OOF predictions ===
meta_model = LinearRegression()
meta_model.fit(X_meta_train, y_train)

# === 4. Predict on holdout test set ===
meta_pred_test = meta_model.predict(X_meta_test)

# === 5. Evaluate final stacked model ===
def evaluate(y_true, y_pred):
    return {
        "R²": r2_score(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_true, y_pred))
    }

stacked_cv_results = evaluate(y_test, meta_pred_test)
print("Cross-Validated Stacked Ensemble Performance:")
for k, v in stacked_cv_results.items():
    print(f"{k}: {v:.4f}")


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# === Set global font ===
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['legend.fontsize'] = 14

# === Prepare data for plotting ===
df_plot = pd.DataFrame({
    "timestamp": ts_test,
    "True Stage": y_test,
    "Simple Avg (XGB + RF + LGB)": avg_all_3,
    "Weighted Avg (0.4 RF + 0.4 LGB + 0.2 XGB)": weighted_avg,
    "Stacked Ensemble (All 3)": stacked_all_3
}).sort_values("timestamp")

# === Create subplots ===
fig, axs = plt.subplots(3, 1, figsize=(15, 12), sharex=True)

# Plot 1: Simple Average
axs[0].plot(df_plot["timestamp"], df_plot["True Stage"], label="True Stage", color="black", linewidth=2)
axs[0].plot(df_plot["timestamp"], df_plot["Simple Avg (XGB + RF + LGB)"], label="Simple Average", color="royalblue", linewidth=2, alpha=0.7)
axs[0].set_title("Simple Average (XGB + RF + LGB) vs. True Stage")
axs[0].set_ylabel("Stage (cm)", fontsize=14)
axs[0].legend()
axs[0].tick_params(axis='both', labelsize=14)
axs[0].grid(True)

# Plot 2: Weighted Average
axs[1].plot(df_plot["timestamp"], df_plot["True Stage"], label="True Stage", color="black", linewidth=2)
axs[1].plot(df_plot["timestamp"], df_plot["Weighted Avg (0.4 RF + 0.4 LGB + 0.2 XGB)"], label="Weighted Average", color="darkorange", linewidth=2, alpha=0.7)
axs[1].set_title("Weighted Average (0.4 RF + 0.4 LGB + 0.2 XGB) vs. True Stage")
axs[1].set_ylabel("Stage (cm)", fontsize=14)
axs[1].legend()
axs[1].tick_params(axis='both', labelsize=14)
axs[1].grid(True)

# Plot 3: Stacked Ensemble
axs[2].plot(df_plot["timestamp"], df_plot["True Stage"], label="True Stage", color="black", linewidth=2)
axs[2].plot(df_plot["timestamp"], df_plot["Stacked Ensemble (All 3)"], label="Stacked Ensemble", color="seagreen", linewidth=2, alpha=0.7)
axs[2].set_title("Stacked Ensemble (XGB + RF + LGB) vs. True Stage")
axs[2].set_xlabel("Timestamp", fontsize=14)
axs[2].set_ylabel("Stage (cm)", fontsize=14)
axs[2].legend()
axs[2].tick_params(axis='both', labelsize=14)
axs[2].grid(True)

plt.tight_layout()

# === Save the figure ===
plt.savefig("ensemble_stage_predictions.png", dpi=300, bbox_inches='tight')

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# === Create subplots ===
fig, axs = plt.subplots(1, 3, figsize=(18, 5), sharey=True)

# Titles and predictions
titles = [
    "Simple Avg (XGB + RF + LGB)",
    "Weighted Avg (0.4 RF + 0.4 LGB + 0.2 XGB)",
    "Stacked Ensemble (All 3)"
]
predictions = [avg_all_3, weighted_avg, stacked_all_3]

# === Plot each ensemble prediction vs. actual stage ===
for ax, title, pred in zip(axs, titles, predictions):
    # Scatter plot
    ax.scatter(y_test, pred, alpha=0.7, edgecolors='k')

    # 1:1 Line
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label="1:1 Line")

    # Regression line
    coeffs = np.polyfit(y_test, pred, deg=1)
    reg_line = np.poly1d(coeffs)
    ax.plot(y_test, reg_line(y_test), color='blue', linewidth=2, label="Regression Line")

    # Labels and formatting
    ax.set_xlabel("Actual Stage (cm)", fontsize=14)
    ax.set_title(title, fontsize=15)
    ax.tick_params(axis='both', labelsize=14)
    ax.legend()

# Shared Y-axis label
axs[0].set_ylabel("Predicted Stage (cm)", fontsize=14)

# Main title
fig.suptitle("Actual vs Predicted Stage (Top Ensemble Models)", fontsize=16)

# Layout and save
plt.tight_layout(rect=[0, 0, 1, 0.93])
plt.savefig("ensemble_scatter_top3.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error
import matplotlib.pyplot as plt

# === Calculate Residuals ===
resid_simple = y_test - avg_all_3
resid_weighted = y_test - weighted_avg
resid_stacked = y_test - stacked_all_3

# === Create subplots ===
fig, axs = plt.subplots(1, 3, figsize=(18, 5), sharey=True)

# Titles and predictions
titles = [
    "Simple Avg (XGB + RF + LGB)",
    "Weighted Avg (0.4 RF + 0.4 LGB + 0.2 XGB)",
    "Stacked Ensemble (All 3)"
]
predictions = [avg_all_3, weighted_avg, stacked_all_3]
residuals = [resid_simple, resid_weighted, resid_stacked]

# === Plot residuals ===
for ax, title, pred, resid in zip(axs, titles, predictions, residuals):
    ax.scatter(pred, resid, alpha=0.7, edgecolors='k')
    ax.axhline(0, color='red', linestyle='--')
    ax.set_xlabel("Predicted Stage (cm)", fontsize=14)
    ax.set_title(
        f"{title}\n$R^2$ = {r2_score(y_test, pred):.3f}, MAE = {mean_absolute_error(y_test, pred):.4f}",
        fontsize=13
    )
    ax.tick_params(axis='both', labelsize=12)

# Shared Y-axis label
axs[0].set_ylabel("Residual (Actual - Predicted) (cm)", fontsize=14)

# Layout and save
plt.tight_layout()
plt.savefig("ensemble_residuals_top3.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# === Calculate Prediction Errors ===
error_simple = avg_all_3 - y_test
error_weighted = weighted_avg - y_test
error_stacked = stacked_all_3 - y_test

# === Create histograms of prediction errors ===
fig, axs = plt.subplots(1, 3, figsize=(18, 5), sharey=True)

# Titles and errors
titles = [
    "Simple Avg (XGB + RF + LGB)",
    "Weighted Avg (0.4 RF + 0.4 LGB + 0.2 XGB)",
    "Stacked Ensemble (All 3)"
]
errors = [error_simple, error_weighted, error_stacked]

for ax, name, error in zip(axs, titles, errors):
    ax.hist(error, bins=40, color='skyblue', edgecolor='black')
    ax.axvline(0, color='red', linestyle='--')
    ax.set_title(f"{name} Error Histogram", fontsize=14)
    ax.set_xlabel("Prediction Error (cm)", fontsize=14)
    ax.tick_params(axis='both', labelsize=14)

# Shared Y-axis label
axs[0].set_ylabel("Frequency", fontsize=14)

# Layout and optional save
plt.tight_layout()
plt.savefig("ensemble_error_histogram_top3.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
true_stage = y_test.values
pred_simple = avg_all_3
pred_weighted = weighted_avg
pred_stacked = stacked_all_3

# Summary metrics
summary_metrics = pd.DataFrame({
    "Model": ["Simple Avg (2 models)", "Weighted Avg", "Stacked Ensemble (3 models)"],
    "R²": [
        r2_score(true_stage, pred_simple),
        r2_score(true_stage, pred_weighted),
        r2_score(true_stage, pred_stacked)
    ],
    "MAE": [
        mean_absolute_error(true_stage, pred_simple),
        mean_absolute_error(true_stage, pred_weighted),
        mean_absolute_error(true_stage, pred_stacked)
    ]
})

summary_metrics

In [ ]:
# Bland-Altman plot function
def bland_altman_plot(actual, predicted, model_name):
    mean_values = (actual + predicted) / 2
    diff = actual - predicted
    mean_diff = np.mean(diff)
    std_diff = np.std(diff)
    upper_limit = mean_diff + 1.96 * std_diff
    lower_limit = mean_diff - 1.96 * std_diff

    plt.figure(figsize=(8, 5))
    plt.scatter(mean_values, diff, alpha=0.6, edgecolors='k', color='cornflowerblue')
    plt.axhline(mean_diff, color='red', linestyle='--', label=f'Mean Diff: {mean_diff:.3f}')
    plt.axhline(upper_limit, color='grey', linestyle='--', label=f'+1.96 SD: {upper_limit:.3f}')
    plt.axhline(lower_limit, color='grey', linestyle='--', label=f'-1.96 SD: {lower_limit:.3f}')
    plt.fill_between(mean_values, lower_limit, upper_limit, color='grey', alpha=0.1)

    plt.title(f'Bland-Altman Plot ({model_name})', fontsize=14)
    plt.xlabel('Mean of Actual and Predicted Stage (cm)', fontsize=14)
    plt.ylabel('Residual (Actual - Predicted) (cm)', fontsize=14)
    plt.legend(fontsize=11)
    plt.grid(True)
    plt.tick_params(axis='both', labelsize=14)
    plt.tight_layout()

    # Save figure
    plt.savefig(f"bland_altman_{model_name.lower().replace(' ', '_')}.png", dpi=300, bbox_inches='tight')
    plt.show()

# Run for all three models
for model_name, y_pred in zip(
    ["Simple Avg (SVR + RF)", "Weighted Avg", "Stacked Ensemble (All 3)"],
    [avg_all_3, weighted_avg, stacked_all_3]
):
    bland_altman_plot(y_test, y_pred, model_name)

In [ ]:
# === Function to plot residuals over time ===
def plot_residuals_over_time(residuals_dict, timestamps):
    plt.figure(figsize=(14, 5))

    for label, resids in residuals_dict.items():
        plt.plot(timestamps, resids, label=label, alpha=0.5)

    plt.axhline(0, color='red', linestyle='--')
    plt.title("Residuals Over Time", fontsize=15)
    plt.xlabel("Timestamp", fontsize=13)
    plt.ylabel("Residual (Actual - Predicted) (cm)", fontsize=14)
    plt.legend(fontsize=11)
    plt.grid(True)
    plt.tick_params(axis='both', labelsize=14)
    plt.tight_layout()

    # Save figure
    plt.savefig("ensemble_timeseries_residuals_top3.png", dpi=300, bbox_inches='tight')
    plt.show()

# === Ensure timestamps are aligned with y_test ===
timestamps_sorted = ts_test.sort_values().reset_index(drop=True)

# === Residuals dictionary structure for top 3 ensemble models ===
residuals_top3 = {
    "Simple Avg (XGB + RF + LGB)": avg_all_3 - y_test,
    "Weighted Avg (0.4 RF + 0.4 LGB + 0.2 XGB)": weighted_avg - y_test,
    "Stacked Ensemble (All 3)": stacked_all_3 - y_test
}

# === Plot ===
plot_residuals_over_time(residuals_dict=residuals_top3, timestamps=timestamps_sorted)


In [ ]:
# === Compute residuals ===
residual_simple = y_test - avg_all_3
residual_weighted = y_test - weighted_avg
residual_stacked = y_test - stacked_all_3

# === Create subplots ===
fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharey=True)

# === Simple Avg plot ===
sns.kdeplot(
    residual_simple, ax=axes[0], fill=True,
    color="#1f77b4", alpha=0.5, linewidth=1
)
axes[0].axvline(x=0, color='black', linestyle='--', linewidth=1.2)
axes[0].set_title("Simple Avg (XGB + RF + LGB)", fontsize=14)
axes[0].set_xlabel("Prediction Error (cm)", fontsize=14)
axes[0].set_ylabel("Density", fontsize=14)
axes[0].grid(True, linestyle="--", alpha=0.3)

# === Weighted Avg plot ===
sns.kdeplot(
    residual_weighted, ax=axes[1], fill=True,
    color="#ff7f0e", alpha=0.4, linewidth=1
)
axes[1].axvline(x=0, color='black', linestyle='--', linewidth=1.2)
axes[1].set_title("Weighted Avg (0.4 RF + 0.4 LGB + 0.2 XGB)", fontsize=14)
axes[1].set_xlabel("Prediction Error (cm)", fontsize=14)
axes[1].grid(True, linestyle="--", alpha=0.3)

# === Stacked Ensemble plot ===
sns.kdeplot(
    residual_stacked, ax=axes[2], fill=True,
    color="#2ca02c", alpha=0.3, linewidth=1
)
axes[2].axvline(x=0, color='black', linestyle='--', linewidth=1.2)
axes[2].set_title("Stacked Ensemble (All 3)", fontsize=14)
axes[2].set_xlabel("Prediction Error (cm)", fontsize=14)
axes[2].grid(True, linestyle="--", alpha=0.3)

# === Final layout ===
plt.tight_layout()
plt.savefig("ensemble_error_kde_top3_separate.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
from scipy.stats import skew, kurtosis

# Generate example residuals (replace with actual residuals in real case)
np.random.seed(42)
residual_simple = np.random.normal(loc=0, scale=0.1, size=100)
residual_weighted = np.random.normal(loc=0, scale=0.08, size=100)
residual_stacked = np.random.normal(loc=0, scale=0.07, size=100)

# Create summary dataframe
summary_stats = pd.DataFrame({
    "Model": ["Simple Avg (2)", "Weighted Avg", "Stacked Ensemble (3)"],
    "Mean Error": [residual_simple.mean(), residual_weighted.mean(), residual_stacked.mean()],
    "Std Dev": [residual_simple.std(), residual_weighted.std(), residual_stacked.std()],
    "Skewness": [skew(residual_simple), skew(residual_weighted), skew(residual_stacked)],
    "Kurtosis": [kurtosis(residual_simple), kurtosis(residual_weighted), kurtosis(residual_stacked)]
})
summary_stats

In [ ]:
import joblib

joblib.dump(lgb_model, "lgb_model.pkl")
joblib.dump(rf_tuned, "rf_model.pkl")
joblib.dump(xgb_model, "xgb_model.pkl")
meta_model_3 = LinearRegression().fit(stack_all_3, y_test)
joblib.dump(meta_model_3, "stacked_meta_model.pkl")


In [ ]:
import joblib
import numpy as np
import pandas as pd

# === Load Models ===
lgb_model = joblib.load("lgb_model.pkl")
rf_model = joblib.load("rf_model.pkl")         # rf_tuned saved here
xgb_model = joblib.load("xgb_model.pkl")
meta_model_3 = joblib.load("stacked_meta_model.pkl")

# === Prediction Function (ROI2 only) ===
def predict_stage_from_roi2(roi2):
    """
    Predict water stage from ROI2 using stacked ensemble model.
    """
    features = np.array([[roi2]])

    # Base model predictions
    pred_lgb = lgb_model.predict(features)[0]
    pred_rf  = rf_model.predict(features)[0]
    pred_xgb = xgb_model.predict(features)[0]

    # Meta model prediction
    stacked_input = np.array([[pred_lgb, pred_rf, pred_xgb]])
    stage_prediction = meta_model_3.predict(stacked_input)[0]

    return stage_prediction

# === Predict for Full Dataset ===
X_all = iqr_filtered_df[["ROI2"]].copy()
timestamps_all = iqr_filtered_df["timestamp"]

# Base model predictions
y_pred_lgb_all = lgb_model.predict(X_all)
y_pred_rf_all  = rf_model.predict(X_all)
y_pred_xgb_all = xgb_model.predict(X_all)

# Meta-model stacked prediction
stacked_input_all = np.vstack([y_pred_lgb_all, y_pred_rf_all, y_pred_xgb_all]).T
stage_pred_all = meta_model_3.predict(stacked_input_all)

# Output DataFrame
predicted_stage_df = pd.DataFrame({
    "timestamp": timestamps_all.values,
    "ROI2": X_all["ROI2"].values,
    "predicted_stage": stage_pred_all
})

predicted_stage_df.head()


In [ ]:
import matplotlib.pyplot as plt

# Sort by timestamp to ensure clean time series plot
plot_df = predicted_stage_df.copy()
plot_df["timestamp"] = timestamps_all.values  # Ensure timestamp column exists
plot_df["actual_stage"] = iqr_filtered_df["Stage"].values  # Replace with correct actual stage column if needed
plot_df = plot_df.sort_values("timestamp")

# Plot
plt.figure(figsize=(15, 6))
plt.plot(plot_df["timestamp"], plot_df["actual_stage"], label="Actual Stage", color="black", linewidth=2)
plt.plot(plot_df["timestamp"], plot_df["predicted_stage"], label="Predicted Stage (Stacked Ensemble)", color="seagreen", linestyle='--', linewidth=2, alpha=0.7)

# Labels and formatting
plt.title("Predicted vs. Actual Water Stage", fontsize=16)
plt.xlabel("Timestamp", fontsize=14)
plt.ylabel("Stage (cm)", fontsize=14)
plt.legend(fontsize=12)
plt.grid(True, linestyle="--", alpha=0.5)
plt.xticks()
plt.tight_layout()

# Save and show
plt.savefig("predicted_vs_actual_stage.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Keep only positive discharge and reasonable stage values
filtered = iqr_filtered_df[(iqr_filtered_df["Discharge"] > 0) & (iqr_filtered_df["Stage"] > 0.1)].copy()

# Extract arrays for curve fitting
h = filtered["Stage"].values
Q = filtered["Discharge"].values


In [ ]:
from scipy.optimize import curve_fit
import numpy as np

# Rating curve function
def rating_curve(h, a, b, h0):
    return a * np.maximum((h - h0), 0) ** b

# Initial guess and curve fitting
initial_guess = (1.0, 2.0, 0.5)
popt, pcov = curve_fit(rating_curve, h, Q, p0=initial_guess, maxfev=10000)

# Extract fitted parameters
a, b, h0 = popt
print(f"Fitted Rating Curve Parameters:\na = {a:.4f}, b = {b:.4f}, h0 = {h0:.4f}")


In [ ]:
import matplotlib.pyplot as plt

h_fit = np.linspace(h.min(), h.max(), 200)
Q_fit = rating_curve(h_fit, *popt)

plt.figure(figsize=(10, 6))
plt.scatter(h, Q, label="Observed Discharge", color="black", alpha=0.6)
plt.plot(h_fit, Q_fit, label="Fitted Rating Curve", color="red", linewidth=2)
plt.xlabel("Stage (cm)")
plt.ylabel("Discharge (cms)")
plt.title("Stage-Discharge Rating Curve")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("rating_curve.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# === 1. Define rating curve function ===
def rating_curve(h, a=0.0310, b=1.2041, h0=6.6794):
    return a * np.maximum((h - h0), 0) ** b

# === 2. Apply to ensemble-predicted stage ===
predicted_stage_df["predicted_discharge"] = rating_curve(predicted_stage_df["predicted_stage"])

# === 3. Merge with actual discharge for comparison ===
pred_discharge_df = pd.merge(
    predicted_stage_df,
    iqr_filtered_df[["timestamp", "Discharge"]],
    on="timestamp",
    how="inner"
).dropna(subset=["Discharge", "predicted_discharge"])


In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

# === Filter entire DataFrame where Discharge is valid ===
valid_df = pred_discharge_df[pred_discharge_df["Discharge"] != -9999.0].copy()

# Now extract all aligned values from the same filtered DataFrame
y_true = valid_df["Discharge"]
y_pred = valid_df["predicted_discharge"]
timestamps = valid_df["timestamp"]

# === Extract valid true and predicted values ===
y_true = valid_df["Discharge"]
y_pred = valid_df["predicted_discharge"]

# === Compute Evaluation Metrics ===
discharge_metrics = {
    "R²": r2_score(y_true, y_pred),
    "RMSE": np.sqrt(mean_squared_error(y_true, y_pred)),
    "MAE": mean_absolute_error(y_true, y_pred)
}

# === Display Results ===
for k, v in discharge_metrics.items():
    print(f"{k}: {v:.4f}")


In [ ]:
from scipy.signal import savgol_filter
y_pred_smooth = savgol_filter(y_pred, window_length=21, polyorder=2)

plt.figure(figsize=(15, 6))

# Plot actual and predicted discharge
plt.plot(timestamps, y_true, label="Observed Discharge", linewidth=2)
plt.plot(timestamps, y_pred_smooth, label="Predicted Discharge", linestyle="--", linewidth=2, alpha=0.9)

# Labels and title
plt.title("Predicted vs. Observed Discharge", fontsize=16)
plt.xlabel("Timestamp", fontsize=14)
plt.ylabel("Discharge (cms)", fontsize=14)

# Legend and grid
plt.legend(fontsize=12)
plt.grid(True, linestyle="--", alpha=0.5)

# Save and show
plt.tight_layout()
plt.savefig("predicted_vs_actual_discharge.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Remove -9999 from either column
clean_df = pred_discharge_df[
    (pred_discharge_df["Discharge"] != -9999) &
    (pred_discharge_df["predicted_discharge"] != -9999)
]

# Prepare values
x = clean_df["Discharge"].values.reshape(-1, 1)  # Observed
y = clean_df["predicted_discharge"].values       # Predicted

# Fit regression line
reg_model = LinearRegression()
reg_model.fit(x, y)
y_pred_line = reg_model.predict(x)

# Create regression equation string
slope = reg_model.coef_[0]
intercept = reg_model.intercept_
reg_eqn = f"y = {slope:.2f}x + {intercept:.2f}"

# Plot
plt.figure(figsize=(8, 7))
sns.scatterplot(x=clean_df["Discharge"], y=clean_df["predicted_discharge"],
                color='royalblue', edgecolor='k', alpha=0.6, s=60, label="Predictions")

# Plot 1:1 line
lims = [min(x.min(), y.min()), max(x.max(), y.max())]
plt.plot(lims, lims, 'r--', linewidth=2, label="1:1 Line")

# Regression line with equation
plt.plot(x.flatten(), y_pred_line, color="seagreen", linestyle='-', linewidth=2, label=f"Regression Line\n({reg_eqn})")

# Labels and styling
plt.xlabel("Observed Discharge (cms)", fontsize=14)
plt.ylabel("Predicted Discharge (cms)", fontsize=14)
plt.title("Observed vs Predicted Discharge", fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend(fontsize=12)
plt.tight_layout()

# Save
plt.savefig("discharge_scatter.png", dpi=300, bbox_inches='tight')
plt.show()
